# YOLO11-OBB Finetuning for Engineering Drawing Callouts (v2)

Train a YOLO11s-OBB model to detect 4 callout types:
- **Hole** (idx 0) — diameter callouts
- **TappedHole** (idx 1) — thread callouts
- **Fillet** (idx 4) — radius callouts
- **Chamfer** (idx 5) — chamfer callouts

**Monitoring:** TensorBoard (run `%tensorboard --logdir runs` in a cell)
**Model saving:** Hugging Face Hub

**Requirements:** A100 GPU, Roboflow API key, HuggingFace token

## Cell 1: Install Dependencies & Check GPU

In [ ]:
!pip install -q ultralytics roboflow huggingface_hub

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    mem = getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0)
    print(f"Memory: {mem / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. Training will be very slow.")

## Cell 2: Clone Repo & Setup

In [ ]:
import os

REPO_DIR = "/content/AI-Drawing-Inspector"

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/skaumbdoallsaws-coder/AI-Drawing-Inspector.git {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

## Cell 3: Download Dataset from Roboflow

In [ ]:
from roboflow import Roboflow

# --- Roboflow credentials ---
ROBOFLOW_API_KEY = "IHyhfpN5KngIAXiN5MdM"
ROBOFLOW_WORKSPACE = "ai-drawing-inspector"
ROBOFLOW_PROJECT = "ai-inspector-callout-detection"
ROBOFLOW_VERSION = 3  # Updated to version 3 with expanded dataset
# -----------------------------

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
dataset = project.version(ROBOFLOW_VERSION).download("yolov8-obb", location="/content/dataset")

DATASET_ROOT = "/content/dataset"
print(f"Dataset downloaded to: {DATASET_ROOT}")
print(f"Train images: {len(os.listdir(os.path.join(DATASET_ROOT, 'train', 'images')))}")

# Check if valid split exists
valid_img_dir = os.path.join(DATASET_ROOT, "valid", "images")
if os.path.isdir(valid_img_dir) and len(os.listdir(valid_img_dir)) > 0:
    print(f"Val images:   {len(os.listdir(valid_img_dir))}")
else:
    print("No valid split from Roboflow — will create one next.")

## Cell 3b: Create Train/Valid Split (if needed)

Roboflow may export all images into train/. This cell splits 20% into valid/,
ensuring all 4 classes are represented in the validation set.

In [ ]:
import shutil, random
from collections import defaultdict

train_img_dir = os.path.join(DATASET_ROOT, "train", "images")
train_lbl_dir = os.path.join(DATASET_ROOT, "train", "labels")
valid_img_dir = os.path.join(DATASET_ROOT, "valid", "images")
valid_lbl_dir = os.path.join(DATASET_ROOT, "valid", "labels")

# Only split if valid is empty or missing
needs_split = True
if os.path.isdir(valid_img_dir):
    valid_files = [f for f in os.listdir(valid_img_dir) if f.endswith(('.jpg','.png','.jpeg'))]
    if len(valid_files) > 0:
        needs_split = False
        print(f"Valid split already exists ({len(valid_files)} images). Skipping.")

if needs_split:
    os.makedirs(valid_img_dir, exist_ok=True)
    os.makedirs(valid_lbl_dir, exist_ok=True)

    # Roboflow alphabetical: 0=Chamfer, 1=Fillet, 2=Hole, 3=TappedHole
    RF_CLASS_MAP = {0: "Chamfer", 1: "Fillet", 2: "Hole", 3: "TappedHole"}

    # Parse classes per image
    image_classes = {}
    for img in sorted(os.listdir(train_img_dir)):
        if not img.endswith(('.jpg', '.png', '.jpeg')):
            continue
        base = os.path.splitext(img)[0]
        lbl_path = os.path.join(train_lbl_dir, base + '.txt')
        classes = set()
        if os.path.exists(lbl_path):
            with open(lbl_path) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 9:
                        classes.add(RF_CLASS_MAP.get(int(parts[0]), 'unk'))
        image_classes[img] = classes

    random.seed(42)
    val_set = set()

    # Ensure each class is represented in valid
    for target_cls in ["Chamfer", "TappedHole", "Fillet", "Hole"]:
        candidates = [img for img, cls in image_classes.items()
                      if target_cls in cls and img not in val_set]
        random.shuffle(candidates)
        for img in candidates[:3]:
            val_set.add(img)

    # Fill to ~20% total
    target_val = max(16, int(len(image_classes) * 0.2))
    remaining = [img for img in image_classes if img not in val_set]
    random.shuffle(remaining)
    while len(val_set) < target_val and remaining:
        val_set.add(remaining.pop())

    # Move files
    for img in sorted(val_set):
        base = os.path.splitext(img)[0]
        shutil.move(os.path.join(train_img_dir, img), os.path.join(valid_img_dir, img))
        lbl_file = base + '.txt'
        lbl_src = os.path.join(train_lbl_dir, lbl_file)
        if os.path.exists(lbl_src):
            shutil.move(lbl_src, os.path.join(valid_lbl_dir, lbl_file))

    train_count = len([f for f in os.listdir(train_img_dir) if f.endswith(('.jpg','.png'))])
    valid_count = len([f for f in os.listdir(valid_img_dir) if f.endswith(('.jpg','.png'))])
    print(f"Split complete: {train_count} train / {valid_count} valid")

## Cell 4: Remap Class Indices

Roboflow exports classes alphabetically (Chamfer=0, Fillet=1, Hole=2, TappedHole=3).
Our pipeline uses classes.py indices (Hole=0, TappedHole=1, Fillet=4, Chamfer=5).
This cell remaps all label files.

In [ ]:
import sys
sys.path.insert(0, REPO_DIR)

from ai_inspector.fine_tuning.data_generator import (
    remap_labels,
    ROBOFLOW_TO_CLASSES_PY,
)

print("Remapping class indices:")
print(f"  Roboflow -> classes.py: {ROBOFLOW_TO_CLASSES_PY}")
print()

for split in ["train", "valid", "test"]:
    label_dir = os.path.join(DATASET_ROOT, split, "labels")
    if os.path.isdir(label_dir):
        result = remap_labels(label_dir, ROBOFLOW_TO_CLASSES_PY)
        print(f"  {split}: {result['files']} files, {result['annotations']} annotations")
    else:
        print(f"  {split}: no labels directory")

# Verify a sample
train_labels = os.path.join(DATASET_ROOT, "train", "labels")
sample_file = sorted(os.listdir(train_labels))[0]
print(f"\nSample label ({sample_file}):")
with open(os.path.join(train_labels, sample_file)) as f:
    for line in f.readlines()[:3]:
        cls_idx = int(line.split()[0])
        from ai_inspector.detection.classes import IDX_TO_CLASS
        cls_name = IDX_TO_CLASS.get(cls_idx, f"UNKNOWN({cls_idx})")
        print(f"  class {cls_idx} = {cls_name}")

## Cell 5: Generate dataset.yaml

In [ ]:
from ai_inspector.fine_tuning.data_generator import generate_dataset_yaml

yaml_path = os.path.join(DATASET_ROOT, "dataset.yaml")
generate_dataset_yaml(yaml_path, DATASET_ROOT)

print(f"Generated: {yaml_path}")
print()
with open(yaml_path) as f:
    print(f.read())

## Cell 6: Train YOLO11s-OBB

**Model:** yolo11s-obb (small) — more capacity than nano for better accuracy

**Monitoring:** TensorBoard — run the cell below to launch, then run training

Drawing-safe augmentation settings:
- No horizontal/vertical flip (text becomes unreadable)
- No hue/saturation changes (drawings are monochrome)
- Minimal rotation (callouts are axis-aligned)
- Reduced mosaic probability

In [ ]:
# Launch TensorBoard to monitor training in real-time
# Run this cell FIRST, then run the training cell below
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
from ultralytics import YOLO
import os

# --- TensorBoard ---
# Run this in a separate cell BEFORE training to monitor live:
#   %load_ext tensorboard
#   %tensorboard --logdir runs
# -------------------

# Load pretrained model (small = more capacity than nano)
model = YOLO("yolo11s-obb.pt")

# Train with drawing-safe augmentation
results = model.train(
    data=os.path.join(DATASET_ROOT, "dataset.yaml"),
    epochs=150,
    batch=32,
    imgsz=1024,
    amp=True,
    device=0,
    project=os.path.join(os.getcwd(), "runs", "obb"),
    name="callout_v2",
    exist_ok=True,
    # Drawing-safe augmentation
    flipud=0.0,        # No vertical flip
    fliplr=0.0,        # No horizontal flip
    hsv_h=0.0,         # No hue shift (monochrome)
    hsv_s=0.0,         # No saturation shift
    hsv_v=0.1,         # Tiny brightness variation
    degrees=2.0,       # Minimal rotation (callouts are axis-aligned)
    mosaic=0.3,        # Reduced mosaic (preserve spatial context)
    scale=0.3,         # Moderate scale augmentation
    translate=0.1,     # Small translation
    # Performance
    workers=4,
    patience=30,       # Early stopping
    save_period=25,    # Save checkpoint every 25 epochs
    verbose=True,
)

# Store the actual weights path for subsequent cells
import glob
BEST_WEIGHTS = glob.glob(os.path.join(os.getcwd(), "runs", "obb", "**", "best.pt"), recursive=True)[-1]
print(f"\nTraining complete!")
print(f"Best weights: {BEST_WEIGHTS}")

## Cell 7: Validate & Print Per-Class mAP

In [ ]:
# Load best model (use path found after training)
best_model = YOLO(BEST_WEIGHTS)

# Run validation
val_results = best_model.val(
    data=os.path.join(DATASET_ROOT, "dataset.yaml"),
    imgsz=1024,
    batch=32,
    device=0,
)

# Print per-class results
print("\n" + "="*50)
print("VALIDATION RESULTS")
print("="*50)
print(f"mAP50:     {val_results.box.map50:.4f}")
print(f"mAP50-95:  {val_results.box.map:.4f}")
print()

# Per-class mAP50
from ai_inspector.detection.classes import IDX_TO_CLASS
print("Per-class mAP50:")
if hasattr(val_results.box, 'ap50') and val_results.box.ap50 is not None:
    for i, ap in enumerate(val_results.box.ap50):
        cls_name = IDX_TO_CLASS.get(i, f"class_{i}")
        if ap > 0:
            print(f"  {cls_name:20s}: {ap:.4f}")
else:
    print("  (per-class AP not available in this format)")

# Pass/Fail check
target_map = 0.5
if val_results.box.map50 >= target_map:
    print(f"\nPASS: mAP50 ({val_results.box.map50:.4f}) >= {target_map}")
else:
    print(f"\nBELOW TARGET: mAP50 ({val_results.box.map50:.4f}) < {target_map}")
    print("Consider: more annotations, more epochs, or larger model")

## Cell 8: Upload Best Weights to Hugging Face Hub

This uploads the trained model to your Hugging Face account for easy sharing and versioning.

In [ ]:
from huggingface_hub import HfApi, login
import shutil
import os

# --- Hugging Face credentials ---
# Option 1: Login interactively (will prompt for token)
login()

# Option 2: Use token directly (uncomment and add your token)
# login(token="hf_YOUR_TOKEN_HERE")
# --------------------------------

# Repository settings
HF_REPO_ID = "skaumbdoallsaws-coder/ai-inspector-callout-detection"  # Change to your username/repo
MODEL_NAME = "callout_v2_yolo11s-obb"

api = HfApi()

# Create repo if it doesn't exist
try:
    api.create_repo(repo_id=HF_REPO_ID, repo_type="model", exist_ok=True)
    print(f"Repository: https://huggingface.co/{HF_REPO_ID}")
except Exception as e:
    print(f"Repo creation note: {e}")

# Upload best weights
print(f"\nUploading {BEST_WEIGHTS}...")
api.upload_file(
    path_or_fileobj=BEST_WEIGHTS,
    path_in_repo=f"{MODEL_NAME}_best.pt",
    repo_id=HF_REPO_ID,
    repo_type="model",
)
print(f"  Uploaded: {MODEL_NAME}_best.pt")

# Upload last weights as backup
last_weights = BEST_WEIGHTS.replace("best.pt", "last.pt")
if os.path.exists(last_weights):
    api.upload_file(
        path_or_fileobj=last_weights,
        path_in_repo=f"{MODEL_NAME}_last.pt",
        repo_id=HF_REPO_ID,
        repo_type="model",
    )
    print(f"  Uploaded: {MODEL_NAME}_last.pt")

# Upload training results CSV
run_dir = os.path.dirname(os.path.dirname(BEST_WEIGHTS))
results_csv = os.path.join(run_dir, "results.csv")
if os.path.exists(results_csv):
    api.upload_file(
        path_or_fileobj=results_csv,
        path_in_repo=f"{MODEL_NAME}_results.csv",
        repo_id=HF_REPO_ID,
        repo_type="model",
    )
    print(f"  Uploaded: {MODEL_NAME}_results.csv")

# Create a simple model card
model_card = f"""---
license: apache-2.0
tags:
  - yolo
  - object-detection
  - obb
  - engineering-drawings
  - callout-detection
---

# {MODEL_NAME}

YOLO11s-OBB model finetuned for engineering drawing callout detection.

## Classes
- **Hole** (idx 0) — diameter callouts
- **TappedHole** (idx 1) — thread callouts  
- **Fillet** (idx 4) — radius callouts
- **Chamfer** (idx 5) — chamfer callouts

## Training
- Base model: yolo11s-obb.pt
- Dataset: Roboflow ai-inspector-callout-detection v3
- Epochs: 150 (with early stopping)
- Image size: 1024
- Augmentation: Drawing-safe (no flip, no hue/sat)

## Usage
```python
from ultralytics import YOLO
model = YOLO("hf://skaumbdoallsaws-coder/ai-inspector-callout-detection/{MODEL_NAME}_best.pt")
results = model("drawing.png")
```
"""

# Save and upload model card
card_path = "/content/README.md"
with open(card_path, "w") as f:
    f.write(model_card)
api.upload_file(
    path_or_fileobj=card_path,
    path_in_repo="README.md",
    repo_id=HF_REPO_ID,
    repo_type="model",
)
print(f"  Uploaded: README.md (model card)")

print(f"\nAll files uploaded to: https://huggingface.co/{HF_REPO_ID}")

## Cell 9: Quick Inference Test

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Pick a validation image
val_img_dir = os.path.join(DATASET_ROOT, "valid", "images")
test_images = sorted(os.listdir(val_img_dir))[:3]

# Color map for classes
CLASS_COLORS = {
    0: "#00FF00",  # Hole - green
    1: "#FF6600",  # TappedHole - orange
    4: "#0099FF",  # Fillet - blue
    5: "#FF00FF",  # Chamfer - magenta
}

fig, axes = plt.subplots(1, min(3, len(test_images)), figsize=(20, 8))
if len(test_images) == 1:
    axes = [axes]

for ax, img_name in zip(axes, test_images):
    img_path = os.path.join(val_img_dir, img_name)
    
    # Run inference
    results = best_model(img_path, imgsz=1024, conf=0.25)
    
    # Plot
    img = Image.open(img_path)
    ax.imshow(img, cmap="gray")
    ax.set_title(img_name[:30], fontsize=10)
    
    if results[0].obb is not None:
        for obb in results[0].obb:
            cls_id = int(obb.cls[0])
            conf = float(obb.conf[0])
            cls_name = IDX_TO_CLASS.get(cls_id, f"cls{cls_id}")
            color = CLASS_COLORS.get(cls_id, "#FFFFFF")
            
            # Get OBB corners
            if hasattr(obb, 'xyxyxyxy'):
                corners = obb.xyxyxyxy[0].cpu().numpy()
                polygon = patches.Polygon(
                    corners, closed=True,
                    linewidth=2, edgecolor=color, facecolor="none"
                )
                ax.add_patch(polygon)
                ax.text(
                    corners[0][0], corners[0][1] - 5,
                    f"{cls_name} {conf:.2f}",
                    color=color, fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.2", facecolor="black", alpha=0.7)
                )
    
    ax.axis("off")

plt.tight_layout()
plt.savefig("inference_preview.png", dpi=150, bbox_inches="tight")
plt.show()
print("Saved: inference_preview.png")

## Cell 10: Integration Test with Pipeline

In [ ]:
from ai_inspector.pipeline import YOLOPipeline
from ai_inspector.config import Config

# Use the finetuned model (use BEST_WEIGHTS from training)
config = Config(yolo_model_path=BEST_WEIGHTS)
pipeline = YOLOPipeline(model_path=config.yolo_model_path, config=config)

# Run on a validation image
test_img = os.path.join(val_img_dir, test_images[0])
result = pipeline.run(test_img)

print(f"Image: {test_images[0]}")
print(f"Detections: {len(result.detections)}")
print(f"Callouts:   {len(result.callouts)}")
print()

for i, callout in enumerate(result.callouts[:10]):
    print(f"  [{i}] {callout.get('calloutType', '?'):15s} | "
          f"raw: {callout.get('raw_text', '?')[:40]}")

print("\nPipeline integration: OK" if result.callouts else "\nNo callouts extracted")